In [1]:
type(open('.gitignore', 'r'))

_io.TextIOWrapper

In [2]:
import influxdb_client, os
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from PyTrasy.SECRETS import INFLUXDB_TOKEN

token = INFLUXDB_TOKEN
org = "freakingrocky"
url = "http://localhost:8086"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)


In [3]:
import time

bucket="test"

write_api = write_client.write_api(write_options=SYNCHRONOUS)
   
for value in range(5):
  point = (
    Point("measurement1")
    .tag("tagname1", "tagvalue1")
    .field("field1", value)
  )
  write_api.write(bucket=bucket, org="freakingrocky", record=point)
  time.sleep(1) # separate points by 1 second


NewConnectionError: <urllib3.connection.HTTPConnection object at 0x000001D5D23174D0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it

In [37]:
import pandas as pd

query_api = write_client.query_api()

query = """from(bucket: "x")
 |> range(start: -10m)
 |> filter(fn: (r) => r._measurement == "measurement1")"""
tables = query_api.query(query, org="freakingrocky")

for table in tables:
  for record in table.records:
    print(record)


ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Encoding', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.6', 'X-Platform-Error-Code': 'not found', 'Date': 'Thu, 09 May 2024 15:07:41 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: b'{"code":"not found","message":"failed to initialize execute state: could not find bucket \\"x\\""}'


In [4]:
print(tables)

NameError: name 'tables' is not defined

In [5]:
pd.DataFrame(tables)

NameError: name 'pd' is not defined

In [6]:
for table in tables:
    for record in table.records:
        print(record)

NameError: name 'tables' is not defined

In [7]:
import polars
data_frame = query_api.query_data_frame(query)
polars_data = polars.from_pandas(data_frame)


NameError: name 'query_api' is not defined

In [8]:
polars_data

NameError: name 'polars_data' is not defined

In [9]:
query_api = write_client.query_api()

query = """from(bucket: "test")
  |> range(start: -10m)
  |> filter(fn: (r) => r._measurement == "measurement1")
  |> mean()"""
data_frame = query_api.query_data_frame(query)
polars_data = polars.from_pandas(data_frame)
polars_data

c:\Python311\Lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "test")
  |> range(start: -10m)
  |> filter(fn: (r) => r._measurement == "measurement1")
  |> mean() |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


NewConnectionError: <urllib3.connection.HTTPConnection object at 0x000001D5FEF70790>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
polars_data

shape: (0, 0)
┌┐
╞╡
└┘

In [3]:
import os
import polars as pl
from tqdm import tqdm
from influxdb_client import InfluxDBClient, Point, WriteOptions
from aiohttp import ClientTimeout
from PyTrasy.SECRETS import INFLUXDB_TOKEN, INFLUX_ORG, INFLUX_URL

# InfluxDB connection details
BUCKET_NAME = "Historical Data"

def csv_loading_func(file_path, symbol_name):
    df = pl.read_csv(file_path)
    df = df.with_columns(
        pl.col('datetime').str.strptime(pl.Datetime, '%Y-%m-%d %H:%M:%S')
    )

    nifty_50_list = [
        "NIFTY 50", "EICHERMOT", "ADANIPORTS", "M&M", "SHRIRAMFIN", "TITAN", "ADANIENT", 
        "CIPLA", "SBILIFE", "BAJAJ-AUTO", "BPCL", "HDFCBANK", "RELIANCE", "ULTRACEMCO", 
        "HINDUNILVR", "BAJFINANCE", "TATAMOTORS", "HDFCLIFE", "AXISBANK", "GRASIM", 
        "JSWSTEEL", "ASIANPAINT", "SUNPHARMA", "BHARTIARTL", "ITC", "BRITANNIA", 
        "HINDALCO", "TATASTEEL", "POWERGRID", "COALINDIA", "MARUTI", "TATACONSUM", 
        "DRREDDY", "DIVISLAB", "BAJAJFINSV", "ICICIBANK", "KOTAKBANK", "ONGC", 
        "INFY", "NTPC", "APOLLOHOSP", "LTIM", "HEROMOTOCO", "NESTLEIND", "SBIN", 
        "INDUSINDBK", "LT", "HCLTECH", "WIPRO", "TCS", "TECHM"]

    # Ensure consistent field types
    df = df.with_columns(pl.col("volume").cast(pl.Int64))  # Cast volume to integer type

    records = []
    for row in df.iter_rows(named=True):
        record = {
            "measurement": "futures_data",
            "tags": {
                "FileSymbol": symbol_name,
                "BaseSymbol": 'NIFTY50' if row['symbol'].upper() in nifty_50_list else 'NIFTY200'
            },
            "fields": {
                "Open": row['open'],
                "High": row['high'],
                "Low": row['low'],
                "Close": row['close'],
                "Volume": row['volume'],
                "Expiry": None,
                "Strike": None,
                "OptionType": None,
                "OI": None
            },
            "time": row['datetime']
        }
        records.append(record)
    return records

def ingest_data(client, bucket, org, records):
    write_api = client.write_api(write_options=WriteOptions(batch_size=500, flush_interval=10_000))
    points = []

    for record in records:
        point = Point(record['measurement'])
        for tag_key, tag_value in record['tags'].items():
            point.tag(tag_key, tag_value)
        for field_key, field_value in record['fields'].items():
            point.field(field_key, field_value)
        point.time(record['time'])
        points.append(point)
    
    write_api.write(bucket=bucket, org=org, record=points)
    write_api.flush()

def process_csv_file(client, bucket, org, file_path, symbol_name):
    records = csv_loading_func(file_path, symbol_name)
    ingest_data(client, bucket, org, records)

def main():
    base_dir = r'C:\Users\freakingrocky\CS Stuff\Tracys\DATA\DATA\NF_200\NF_200_1min\big_data'
    timeout = ClientTimeout(total=7200)
    client = InfluxDBClient(url=INFLUX_URL, token=INFLUXDB_TOKEN, org=INFLUX_ORG, timeout=timeout)
    
    # List all CSV files
    csv_files = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                symbol_name = os.path.splitext(file)[0]  # Extract symbol name from file name
                csv_files.append((file_path, symbol_name))
                
    csv_files.sort()
    index = 0
    while index < len(csv_files):
        if 'TCS' in csv_files[index]:
            break
        index += 1
        
    csv_files = csv_files[index:] + csv_files[:2]
    
    # Process each CSV file with a progress bar
    for file_path, symbol_name in tqdm(csv_files, desc="Processing CSV files"):
        process_csv_file(client, BUCKET_NAME, INFLUX_ORG, file_path, symbol_name)

if __name__ == '__main__':
    main()


Processing CSV files: 100%|██████████| 20/20 [03:38<00:00, 10.93s/it]
